In [1]:
from binance.spot import Spot
import pandas as pd
import requests
import random
import numpy as np
from scipy.stats import norm, chi2,gumbel_r
import scipy.optimize as sco
from statsmodels.stats.correlation_tools import cov_nearest
import datetime

In [2]:
binance_api_key='E90xR7UnO2oeP3lrgC50zkqUuAqyyXs4uSlAGcqIVO9n90ELrFI4gfVI2HGGxxCj'
binance_api_secret='Inls2xYl9FqPi0zlIrucfxG81drx7w6Pv6SzYFxMNoAcwDjlCKug7zUM9zB3lF9h'

In [3]:
binance_api=Spot(binance_api_key,binance_api_secret)

In [4]:
def daterange(start_date, end_date,interval=30):
    for n in range(0,int((end_date - start_date).days),interval):
        yield start_date + datetime.timedelta(n)

In [123]:
class BinanceAPI:
    
    def __init__(self,binance_api_key,binance_api_secret):
        
        self.binance_api_key=binance_api_key
        self.binance_api_secret=binance_api_secret
        
        self.binance_api=Spot(self.binance_api_key,self.binance_api_secret)
        
    def get_market_cap(self,quote="USDT"):

        
        resp = requests.get("https://www.binance.com/bapi/asset/v2/public/asset-service/product/get-products")

        market_cap=pd.DataFrame(resp.json()['data'])
        market_cap=market_cap[market_cap['q']==quote]

        market_cap=market_cap[['an','qn','s','b','q','c','cs']]
        market_cap['c']=market_cap['c'].astype(float)

        market_cap.columns=['Long name','Quote Name','Ticker','Short Name','Quote Short Name','Close','Supply']
        market_cap['Market Cap']=market_cap['Close']*market_cap['Supply']

        market_cap=market_cap.sort_values(by='Market Cap',ascending=False)


        return market_cap


    def get_price(self,ticker_list,date=datetime.datetime.today()):
        
        timestamp_sec=int((date-datetime.timedelta(1)).timestamp()*1000)
        price=pd.DataFrame()

        for ticker in ticker_list:
            try:
                
                temp=pd.DataFrame()
                numeric_columns =  ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                                'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']

                if ticker!='USDTUSDT':
                    
                    data = pd.DataFrame(self.binance_api.klines(ticker,"1d", startTime=timestamp_sec),columns=numeric_columns)
                    data['Close Time']=pd.to_datetime(data['Close Time'], unit='ms')
                    data=data.set_index('Close Time')
                    temp[ticker]=data['Close'].astype(float)
                    price=pd.concat([price,temp[ticker]],axis=1)
                    
                else:
                    
                    price[ticker]=1

            except Exception as e:

                print(ticker +" not retrieved")

                pass

        price.index=pd.to_datetime(price.index).strftime('%Y-%m-%d')

        return price

    def get_inventory(self):
        
        
        ptf=pd.DataFrame(self.binance_api.user_asset())
        ptf['Ticker']=ptf['asset']+"USDT"

        ticker=ptf['Ticker'].to_list()
        ptf=ptf.set_index('Ticker')

        price=self.get_price(ptf.index).T
        price.columns=['Price']

        data=pd.concat([price,ptf],axis=1)
        data['Price in USDT']=data['Price']*(data['free'].astype(float)+data['locked'].astype(float))

        inventory=pd.DataFrame(data['Price in USDT'])
        inventory['Weights in %']=inventory['Price in USDT']/inventory['Price in USDT'].sum()

        inventory.loc['Total']=inventory.sum()

        return inventory.sort_values(by='Weights in %',ascending=False)
    
    def get_positions_history(self,enddate=datetime.datetime.today(),startdate=datetime.datetime(2024,5,5)):
        
        #dt = enddate-datetime.timedelta(days=1)
        #startdate=enddate-datetime.timedelta(30)

        #timestamp_sec = dt.timestamp()
        #timestamp_end = int(timestamp_sec * 1000)
    
        #snapshots=self.binance_api.account_snapshot(type='SPOT',limit=30,endTime=timestamp_end)
        #all_key=snapshots['snapshotVos']  
        
        snapshots=[]

        for date in daterange(startdate,enddate):

            print(date)

            timestamp_sec = date.timestamp()
            timestamp_end = int(timestamp_sec * 1000)
            snapshot=self.binance_api.account_snapshot(type='SPOT',limit=30,startTime=timestamp_end)
            snapshots.extend(snapshot['snapshotVos'])
        
        all_key=(snapshots)
        history={}
        tickers=set()

        for i in range(len(all_key)):

            history[all_key[i]['updateTime']]=all_key[i]['data']['balances']

        for key in history.keys():
            holding=history[key]

            for i in range(len(holding)):
                tickers.add(holding[i]['asset']+"USDT")

        tickers=list(tickers)
        holdings={}

        for key in history.keys():

            temp={}
            date_key=pd.to_datetime(key,unit='ms').strftime('%Y-%m-%d')

            for i in range(len(history[key])):

                temp[history[key][i]['asset']]=float(history[key][i]['free'])+float(history[key][i]['locked'])

            #holdings[key]=temp

            holdings[date_key]=temp

        quantities=pd.DataFrame(holdings).T
        #quantities.index=pd.to_datetime(quantities.index,unit="ms").strftime(date_format='%Y-%m-%d')
        #quantities.index=pd.to_datetime(quantities.index)
        quantities.columns=quantities.columns+'USDT'
        #quantities=quantities.astype(float)


        crypto=quantities.columns

        prices=self.get_price(crypto,startdate)
        
        positions=pd.DataFrame()
        
        for col in crypto:
            
            try:
                positions[col]=quantities[col]*prices.loc[quantities.index][col]
            except Exception as e:
            
                print(col)
                
        return positions

In [128]:
Bi=BinanceAPI(binance_api_key,binance_api_secret)

In [129]:
to=Bi.get_positions_history(enddate=datetime.datetime.today()-datetime.timedelta(1),
                                        startdate=datetime.datetime(2024,5,5))

2024-05-05 00:00:00
2024-06-04 00:00:00
2024-07-04 00:00:00
LDUMAUSDT not retrieved
LDUMAUSDT


In [131]:
to.iloc[:-1]

,ARUSDT,BBUSDT,BNBUSDT,BTCUSDT,ENSUSDT,ETHUSDT,FDUSDUSDT,FETUSDT,INJUSDT,JASMYUSDT,...,RNDRUSDT,SOLUSDT,TNSRUSDT,TRXUSDT,UMAUSDT,USDTUSDT,XRPUSDT,NOTUSDT,EURUSDT,OMUSDT
2024-06-04,60.378788,192.802004,0.000000,780.296640,84.001123,264.187441,0.0,0.057575,0.052706,193.432635,...,43.867154,251.601997,0.0,0.039895,0.136158,0.000000,0.444946,NaN,NaN,NaN
2024-06-11,0.000000,150.476796,0.035608,835.695519,0.000000,242.492097,0.0,0.008936,0.000000,257.969132,...,0.000000,218.742438,0.0,0.012863,0.000000,0.000000,0.000000,NaN,NaN,NaN
2024-06-12,0.000000,153.335047,0.059809,728.597949,0.000000,246.845726,0.0,0.000000,0.000000,408.015809,...,0.000000,227.408397,0.0,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
2024-06-15,0.000000,136.403530,0.027430,706.870007,0.000000,247.443406,0.0,0.007852,0.000000,368.980497,...,0.000000,213.218285,0.0,0.009117,0.000000,0.003498,0.000000,42.858852,NaN,NaN
2024-06-16,0.000000,135.131527,40.262085,711.658236,0.000000,251.303360,0.0,0.000000,0.000000,414.473021,...,0.000000,221.693796,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
2024-06-17,0.000000,109.491954,39.833625,709.816675,0.000000,243.471820,0.0,0.002189,0.000000,390.158425,...,0.000000,209.933972,0.0,0.002668,0.000000,0.000000,0.000000,0.000000,NaN,NaN
2024-06-18,0.000000,101.984647,38.798729,695.631832,0.000000,241.527628,0.0,0.003962,0.000000,347.694409,...,0.000000,201.353011,0.0,0.005275,0.000000,0.000000,0.000000,0.000000,NaN,NaN
2024-06-20,0.000000,98.343229,38.739403,692.372972,0.000000,243.584144,0.0,0.009968,0.000000,310.714157,...,0.000000,195.707417,0.0,0.009000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
2024-06-21,0.000000,98.891935,38.633936,684.619610,0.000000,243.959947,0.0,0.012138,0.000000,314.835092,...,0.000000,197.251472,0.0,0.011104,0.000000,0.000000,0.000000,0.000000,NaN,NaN
2024-06-22,0.000000,99.340877,38.963521,685.883856,0.000000,242.382546,0.0,0.013355,0.000000,341.669840,...,0.000000,195.923223,0.0,0.013232,0.000000,0.000000,0.000000,0.000000,NaN,NaN


In [ ]:
class RiskAnalysis(Portfolio):
    
    
    #This class is used to assess various risk of a portfolio such as Market Risk, VaR
    # and to know which asset could possibly contribute to it
    
    def __init__(self,returns):
        
        self.returns=returns
        super().__init__(returns=returns)
        

    def historical_var(self,weights,last_days=False,Q=5):
        
        #Return Historical VaR on the Past x days at Q confidence interval

        performance=super().portfolio(weights)
        
        if last_days:
            performance=performance[-last_days:]
            
        var=np.percentile(performance,Q)
        cvar=performance[performance<var].mean().values[0]
        
        return var,cvar
    
    
    def parametric_var(self,weights,Q=0.95,stress_factor=1):
        
        #Return parametric VaR, where assets follows a Normal Distribution
        
        intervals=np.arange(Q, 1, 0.0005, dtype=float)
        
        variance=super().variance(weights)*stress_factor
        VaR=variance/np.sqrt(252)*norm(loc =0 , scale = 1).ppf(1-Q)
        CVaR=variance/np.sqrt(252)*norm(loc =0 , scale = 1).ppf(1-intervals).mean()
        
        return VaR,CVaR
        
        
    def multivariate_distribution(self,
                    stress_factor=1.0,
                    iterations=10000):
        
        #Return Multivariate Distribution of a portfolio taking into account potential correlation
        
        num_asset=len(self.returns.columns)
        
        if type(stress_factor)==float:
            
            stress_vec=np.linspace(stress_factor,stress_factor,num_asset)
            
        else:       
            stress_vec=stress_factor
            
        stress_matrix=np.diag(stress_vec)
        stress_matrix=pd.DataFrame(stress_matrix,columns=self.returns.columns,index=self.returns.columns)
        
        stressed_cov=self.returns.cov().dot(stress_matrix)
        mean=self.returns.mean()
        
        multivariate=np.random.multivariate_normal(mean,stressed_cov,iterations)
        
        return multivariate
    
    def gaussian_copula(self,iterations=10000,stress_factor=1.0):
        
       
        randoms=np.random.normal(size=(10000,self.returns.shape[1])).T
        corr_matrix=self.returns.corr()
        
        if type(stress_factor)==float:
            stress_vec=np.linspace(stress_factor,stress_factor,self.returns.shape[1])
            
        else:
            
            stress_vec=stress_factor
        
        if not is_pos_def(corr_matrix):
            corr_matrix=cov_nearest(corr_matrix)
        
        cholesky=np.linalg.cholesky(corr_matrix)
        simulation=np.matmul(cholesky,randoms).T
        simulation=pd.DataFrame(simulation)
        simulation.columns=self.returns.columns

        copula_sample=simulation*self.returns.std()*stress_vec+self.returns.mean()
        
        return copula_sample
    
    def t_copula(self,iterations=10000,stress_factor=1.0):
        

        df=self.returns.shape[1]*self.returns.shape[1]//2+self.returns.shape[1]
        ChiSquared = np.random.chisquare(df=df, size=iterations)

        randoms=np.random.normal(size=(10000,self.returns.shape[1])).T
        corr_matrix=self.returns.corr()
        
        if type(stress_factor)==float:
            stress_vec=np.linspace(stress_factor,stress_factor,self.returns.shape[1])
            
        else:    
            stress_vec=stress_factor
        
        if not is_pos_def(corr_matrix):
            
            corr_matrix=cov_nearest(corr_matrix)
        
        cholesky=np.linalg.cholesky(corr_matrix)
            

        simulation=np.matmul(cholesky,randoms)/np.sqrt(ChiSquared/df)
        simulation=pd.DataFrame(simulation.T)
        simulation.columns=self.returns.columns

        copula_sample=simulation*self.returns.std()*stress_vec+self.returns.mean()
        
        return copula_sample
    
    def monte_carlo(self,spot,horizon=20/250,iterations=100000,stress_factor=1.0):
        
        
        num_asset=len(self.returns.columns)
        #haltons=generate_halton(iterations,num_asset,base=2)
        randoms=np.random.normal(size=(10000,num_asset)).T
        
        # Create a stress matrix to stress the covariance matrix
        
        if type(stress_factor)==float:
            
            stress_vec=np.linspace(stress_factor,stress_factor,num_asset)
            
        else: 
            
            stress_vec=stress_factor
        
        
        #Stress the volatilities of the assets
        
        vol=self.returns.std()*np.sqrt(250)*stress_vec
        
        #Create a diagonal matrix of the stress factors
        
        stress_matrix=np.diag(stress_vec)
        stress_matrix=pd.DataFrame(stress_matrix,columns=self.returns.columns,index=self.returns.columns)
        
        #Find nearest PSD matrix and apply cholesky decomposition to create correaltion effect in Monte Carlo
        
        stressed_cov=self.returns.cov().dot(stress_matrix)
        stressed_std=np.sqrt(np.diag(stressed_cov))
        corr_matrix=stressed_cov/np.outer(stressed_std,stressed_std)
        
        if not is_pos_def(corr_matrix):
            corr_matrix=cov_nearest(corr_matrix)
        
        cholesky=np.linalg.cholesky(corr_matrix)
            
            
        drift=np.exp(-0.5*horizon*vol**2)
        factors=spot*drift
        factors_vec=factors.to_numpy().reshape(num_asset,-1)
                
        simulation=np.matmul(cholesky,randoms).T
        simulation=pd.DataFrame(simulation)
        simulation.columns=self.returns.columns
 
        
        monte_carlo=factors_vec.T*np.exp(simulation.dot(np.diag(vol))*np.sqrt(horizon))
        monte_carlo=pd.DataFrame(monte_carlo)
        monte_carlo.columns=self.returns.columns
        perf_monte_carlo=np.log(monte_carlo/spot)
        
        return monte_carlo,perf_monte_carlo
      

    
    def pca(self,num_components=2):
        
        #Returns the eigen vectors of the covariance matrix
        
        cov_matrix=self.returns.cov()
                
        eig_val, eig_vec=np.linalg.eig(cov_matrix)
        sorted_eig_val=eig_val.argsort()[::-1]
        eig_val=eig_val[sorted_eig_val]
        eig_vec=eig_vec[:,sorted_eig_val]
        eig_val=eig_val[:num_components]
        eig_vec=eig_vec[:,0:num_components]
        
        PC={}
        
        for i in range(eig_vec.shape[1]):
            
            PC["PC" +str(i+1)]=eig_vec[:,i]/eig_vec[:,i].sum()
        
        
        portfolio_components=pd.DataFrame(PC.values(),index=PC.keys(),columns=self.returns.columns).T
        
        return eig_val,eig_vec,portfolio_components
    

    def var_contrib(self,weights):
        
        weights_matrix=np.diag(weights)
        variance_contrib=np.dot(weights_matrix,np.dot(self.returns.cov(),weights_matrix.T))
        
        asset_contrib=variance_contrib.sum(axis=0)    
        diag=np.diag(variance_contrib.diagonal())
        variance_decomposition=np.column_stack([asset_contrib,variance_contrib.diagonal(),(variance_contrib-diag).sum(axis=0)])
        contrib=pd.DataFrame(variance_decomposition,index=self.returns.columns,columns=['Variance Contribution','Idiosyncratic Risk','Correlation'])
        
        weighted_covar=pd.DataFrame(variance_contrib,columns=self.returns.columns,index=self.returns.columns)
        
        return contrib,weighted_covar
    
    def var_contrib_pct(self,weights):
        
        var_contrib=self.var_contrib(weights)[0]
        var_contrib=var_contrib/var_contrib['Variance Contribution'].sum()
        var_contrib.columns=['Variance Contribution in %','Idiosyncratic Risk in %','Correlation in %']
        var_contrib=var_contrib.loc[(var_contrib!=0).any(axis=1)]
        var_contrib=var_contrib.sort_values('Variance Contribution in %', ascending=False)
    
        return var_contrib
    
    def perf_contrib(self,start_weights):
    
        ptf_evolution=((1+self.returns).cumprod()*start_weights)

        last_nav=ptf_evolution.iloc[-1].sum()
        last_row=ptf_evolution.iloc[-1]
        last_weight=last_row/last_nav

        perf=last_row/start_weights
        perf_contrib=(perf-1)*start_weights

        perf_report=pd.concat([perf_contrib,last_weight],axis=1)
        perf_report.columns=['Performance Contribution','Weights']

        return perf_report.dropna()
    
    def perf_contrib_pct(self,weights):
        
        perf_contrib=self.perf_contrib(weights)
        perf_contrib['Performance Contribution']=perf_contrib['Performance Contribution']/perf_contrib['Performance Contribution'].sum()
        perf_contrib.columns=['Performance Contribution in %','Last Weights']
        
        perf_report=perf_contrib.sort_values(by='Performance Contribution in %',ascending=False)
        
        return perf_contrib
    
    def summary(self,weights):
        
        inventory=self.inventory(weights)
        perf_report=self.perf_contrib_pct(weights)
        var_contrib_pct=self.var_contrib_pct(weights)

        report=pd.concat([inventory,perf_report,var_contrib_pct],axis=1)
        
        return report.dropna()
    
    def tracking_error(self,ptf,bench):
        
        excess_return=ptf-bench
        tracking_error=excess_return.std()*np.sqrt(252)
        
        return tracking_error,excess_return